<a href="https://colab.research.google.com/github/basleal22/kaim-week-5/blob/task-3/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/basleal22/kaim-week-5
%cd kaim-week-5
!git switch task-3

Cloning into 'kaim-week-5'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 84 (delta 29), reused 68 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (84/84), 15.07 KiB | 7.53 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/kaim-week-5
Branch 'task-3' set up to track remote branch 'task-3' from 'origin'.
Switched to a new branch 'task-3'


In [4]:
!pip install transformers datasets seqeval
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  # Ensure PyTorch supports GPU


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=d5a2c1cb0898373e80173088ce9c2abbe018e70e5f57c6b4382a98a8e8dfe08a
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency 

In [5]:
import tensorflow as tf

In [6]:
path = '/content/drive/MyDrive/week-5 datas/labeled_telegram_product_price_location_2.csv'

In [7]:
import pandas as pd
data = pd.read_csv(path)

In [8]:
data.shape

(12524, 3)

In [9]:
from transformers import AutoTokenizer,AutoModelForTokenClassification

In [11]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
!pip install datasets


In [13]:
from datasets import Dataset
# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(data)


In [14]:
label_map = {
    'O': 0,  # Outside of any entity
    'B-product': 1,  # Beginning of a product entity
    'I-product': 2,  # Inside a product entity
    'B-price': 3,  # Beginning of a price entity
    'I-price': 4,  # Inside a price entity
    'B-location': 5,  # Beginning of a location entity
    'I-location': 6,  # Inside a location entity
    # Add more labels as needed
}
def tokenize_and_align_labels(examples):
    # Ensure 'Sentence' is a list of strings
    sentences = examples["Sentence"]
    if isinstance(sentences, list):
        sentences = [str(s) for s in sentences]  # Convert all elements to strings
    else:
        sentences = [str(sentences)]  # Wrap single string in a list

    # Tokenize the inputs
    tokenized_inputs = tokenizer(
        sentences,
        truncation=True,
        padding="max_length",
        max_length=512,
        is_split_into_words=False  # Change this if your data is pre-tokenized
    )

    # Process labels similarly...
    all_aligned_labels = []
    for i, labels in enumerate(examples["Labels"]):
        # Ensure labels are a list
        if isinstance(labels, str):
            labels = labels.split()  # Split string labels into a list
        elif not isinstance(labels, list):
            labels = [str(labels)]  # Convert single label to a list

        word_ids = tokenized_inputs.word_ids(i)
        aligned_labels = []
        previous_word_id = None
        for word_id in word_ids:
            if word_id is None:
                aligned_labels.append(-100)
            elif word_id != previous_word_id:
              # Check if word_id is within bounds before accessing labels
              if word_id < len(labels):
                aligned_labels.append(label_map.get(labels[word_id], -100))
              else:
                aligned_labels.append(-100)  # Append -100 for out-of-bounds word_ids
            else:
              # Assign the same label for sub-tokens
              # Check if previous_word_id is within bounds
              if previous_word_id is not None and previous_word_id < len(labels):
                aligned_labels.append(label_map.get(labels[previous_word_id], -100))
              else:
                aligned_labels.append(-100)  # Append -100 if previous_word_id is out of bounds
            previous_word_id = word_id  # Update previous_word_id after processing the current word_id
        all_aligned_labels.append(aligned_labels) # Append the aligned labels for this example to the list

    return {
        "input_ids": tokenized_inputs["input_ids"],
        "attention_mask": tokenized_inputs["attention_mask"],
        "labels": all_aligned_labels,
    }

In [15]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/12524 [00:00<?, ? examples/s]

In [16]:
print(tokenized_dataset.column_names)

['Unnamed: 0', 'Sentence', 'Labels', 'input_ids', 'attention_mask', 'labels']


In [17]:
# Remove unnecessary columns
tokenized_dataset = tokenized_dataset.remove_columns(["Unnamed: 0"])
# Remove the redundant 'Labels' column
tokenized_dataset = tokenized_dataset.remove_columns(["Labels"])
tokenized_dataset = tokenized_dataset.rename_column("labels", "Labels")

# Ensure the dataset has the correct columns
print(tokenized_dataset.column_names)

['Sentence', 'input_ids', 'attention_mask', 'Labels']


In [18]:
import torch
from transformers import AutoModelForSequenceClassification #automatically loads a pre-trained Transformer model specifically tailored for sequence classification tasks.
from peft import LoraConfig, get_peft_model, TaskType #get_peft_model: A function that wraps your base model with LoRA modules.

Choosing a fine -tuning strategy
  using lora from the PEFT

In [19]:
#load the base model
from transformers import AutoModelForTokenClassification #since we are doing token classification

base_model = AutoModelForTokenClassification.from_pretrained(
    model_name,  # e.g., 'xlm-roberta-base'
    num_labels=7  # Number of unique labels for token classification
)
#define LoRA configuration
lora_config = LoraConfig(
       task_type=TaskType.TOKEN_CLS,
       r=8,
       lora_alpha=32,
       lora_dropout=0.1,
       bias="none",
       target_modules=["query", "key", "value", "dense"] # Updated target modules
   )
#convert the base model to a peft model using lora
peft_model = get_peft_model(base_model,lora_config)
#peft_model is now the model you'll train, but only a small fraction of parameters (the lora parameters) will be trainable

#print trainable parameters for confirmaion
trainable_params = 0
all_params = 0
for name, param in peft_model.named_parameters():
  all_params += param.numel()
  if param.requires_grad:
    trainable_params += param.numel()
print(f"trainable params: {trainable_params} || all params: {all_params} || trainable%: {100 * trainable_params / all_params:.2f}%")


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1332487 || all params: 278790926 || trainable%: 0.48%


# Train loop using the trainer API

In [20]:
# ipython-input-42-9c703d3dfa38
from transformers import TrainingArguments, Trainer
# setting train and test sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)  # 80% train, 20% test
train_dataset=train_test_split['train']
val_dataset=train_test_split['test']

# Remove the 'Sentence' column before training
train_dataset = train_dataset.remove_columns(["Sentence"])
val_dataset = val_dataset.remove_columns(["Sentence"])

# Rename 'Labels' column to 'labels' (lowercase)
train_dataset = train_dataset.rename_column("Labels", "labels")
val_dataset = val_dataset.rename_column("Labels", "labels")

print(train_dataset.column_names)

['input_ids', 'attention_mask', 'labels']


In [21]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00


In [22]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/kaim-week-5/lora-xlm-roberta-nlp",
    evaluation_strategy="epoch",    # Evaluates the model after each epoch.
    save_strategy="epoch",          # Saves a checkpoint at the end of every epoch.
    logging_strategy="steps",       # Logs training metrics every few steps.
    logging_steps=200,
    per_device_train_batch_size=8,  # Batch size per device.
    per_device_eval_batch_size=8,   # Evaluation batch size per device.
    num_train_epochs=3,  # Number of full passes through the training data.
    weight_decay=0.01,   # Regularization hyperparameter to help prevent overfitting.
    learning_rate=5e-5,  # Step size for the optimizer.
    push_to_hub=False,   # Disable pushing to Hugging Face Hub.
    report_to="none",    # Disable logging to W&B.
    remove_unused_columns=True,  # Ensure only required columns are passed during training.
    load_best_model_at_end=True,  # Load the best model at the end of training.
    metric_for_best_model="accuracy",  # Use accuracy to determine the best model.
)
# Define a simple accuracy metric

import evaluate
metric_accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)

    # Flatten predictions and labels and remove padding tokens
    predictions = predictions.view(-1)
    labels = [item for sublist in labels for item in sublist]

    # Filter out padding tokens (-100)
    true_predictions = [p.item() for (p, l) in zip(predictions, labels) if l != -100]
    true_labels = [l for l in labels if l != -100]

    return metric_accuracy.compute(predictions=true_predictions, references=true_labels)

# Initialize the Trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<ipython-input-22-462dbbb1ec16>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# Train and evaluate

In [23]:
trainer.train()

# Evaluate on the small evaluation set
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.000000,0.000006,1.000000
2,0.000000,0.000003,1.000000
3,0.000000,0.000002,1.000000


Evaluation Results: {'eval_loss': 6.456377377617173e-06, 'eval_accuracy': 1.0, 'eval_runtime': 87.215, 'eval_samples_per_second': 28.722, 'eval_steps_per_second': 3.6, 'epoch': 3.0}


In [24]:
print(tokenized_dataset[:10])

{'Sentence': ['Coffee Cold Extraction Cup Ice Coffee Maker Portable Cold Brew Bottle Maker Mug Leak Proof Airtight Lid with Mesh Filter Ice Coffee Jar Tea Infuser for Outdoor የአጠቃቀም ዘዴ 1⃣,ለቡና ዱቄት የሚሆን በቂ ቦታ በመተው በውሃ መሙላት ይጀምሩ 2⃣,ውስጥ ባለው ማጣሪያ የሚፈልጉትን የቡና መጠን ይጨምሩ 3⃣,የማጣሪያውን ክዳን አጥብቀው ይዝጉ እና የቡናውን ቦታ በእኩል መጠን ለማከፋፈል ቀስ ብለው ማወዛወዝ 4⃣,ለተወሰኑ ሰአታት ፍሪጅ ውስጥ ማስቀመጥ 5⃣, ቀዝቃዛ ቡናዎን በበረዶ ወይም ከወተት ጋር በመቀላቀል መጠጣት ይችላሉ ዋጋ፦ 900 ብር ውስን ፍሬ ነው ያለው አድራሻ ቁ.1⃣መገናኛ መሰረት ደፋር ሞል ሁለተኛ ፎቅ ቢሮ ቁ. S05/S06 ቁ.2⃣ፒያሳ ጊዮርጊስ አደባባይ ራመት_ታቦር_ኦዳ_ህንፃ 1ኛ ፎቅ ሱቅ ቁ. G1 -107 0902660722 0928460606 ፒያሳ ቅርንጫፍ 0941337070 በTelegram ለማዘዝ ይጠቀሙ @zemencallcenter @zemenexpressadmin ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን https://telegram.me/zemenexpress', 'Coffee Cold Extraction Cup Ice Coffee Maker Portable Cold Brew Bottle Maker Mug Leak Proof Airtight Lid with Mesh Filter Ice Coffee Jar Tea Infuser for Outdoor የአጠቃቀም ዘዴ 1⃣,ለቡና ዱቄት የሚሆን በቂ ቦታ በመተው በውሃ መሙላት ይጀምሩ 2⃣,ውስጥ ባለው ማጣሪያ የሚፈልጉትን የቡና መጠን ይጨምሩ 3⃣,የማጣሪያውን ክዳን አጥብቀው ይዝጉ እና የቡናውን ቦታ በእኩል መጠን ለማከፋፈል ቀስ ብለው 

In [25]:
import torch
device = torch.device("cpu")  # Use CPU for inference
peft_model = peft_model.to(device)  # Move the model to the correct device

def predict_labels(sentence_text):
    """
    Predicts labels for a given sentence using the fine-tuned model.
    Assumes the task is sequence labeling (e.g., NER or other token-level classification).

    Args:
        sentence_text (str): Input sentence for which labels need to be predicted.

    Returns:
        list: Tokens with their predicted labels.
    """
    # Tokenize the input sentence
    encoding = tokenizer(sentence_text, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    # Perform inference
    with torch.no_grad():
        outputs = peft_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Shape: (batch_size, sequence_length, num_labels)

    # Decode logits to label predictions
    predictions = torch.argmax(logits, dim=-1).squeeze(0).tolist()  # Take the highest-scoring label per token

    # Map predictions back to tokens and labels
    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze(0).tolist())
    decoded_predictions = [(token, label_map.get(pred, "O")) for token, pred in zip(tokens, predictions)]

    return decoded_predictions

# Example test sentences
test_sentences = [
    "Addis Ababa is the capital of Ethiopia.",
    "This product costs 50 birr and is available in the market."
]

# Run predictions
for sentence in test_sentences:
    predictions = predict_labels(sentence)
    print(f"Sentence: {sentence}")
    print("Predictions:")
    for token, label in predictions:
        print(f"  {token}: {label}")
    print("\n")

Sentence: Addis Ababa is the capital of Ethiopia.
Predictions:
  <s>: O
  ▁Addis: O
  ▁A: O
  baba: O
  ▁is: O
  ▁the: O
  ▁capital: O
  ▁of: O
  ▁Ethiopia: O
  .: O
  </s>: O


Sentence: This product costs 50 birr and is available in the market.
Predictions:
  <s>: O
  ▁This: O
  ▁product: O
  ▁costs: O
  ▁50: O
  ▁bir: O
  r: O
  ▁and: O
  ▁is: O
  ▁available: O
  ▁in: O
  ▁the: O
  ▁market: O
  .: O
  </s>: O




In [28]:
!zip -r /content/my_folder.zip /content/kaim-week-5/lora-xlm-roberta-nlp

updating: content/kaim-week-5/lora-xlm-roberta-nlp/checkpoint-1253/ (stored 0%)
updating: content/kaim-week-5/lora-xlm-roberta-nlp/checkpoint-1253/training_args.bin (deflated 51%)
updating: content/kaim-week-5/lora-xlm-roberta-nlp/checkpoint-1253/tokenizer_config.json (deflated 76%)
updating: content/kaim-week-5/lora-xlm-roberta-nlp/checkpoint-1253/optimizer.pt (deflated 7%)
updating: content/kaim-week-5/lora-xlm-roberta-nlp/checkpoint-1253/trainer_state.json (deflated 63%)
updating: content/kaim-week-5/lora-xlm-roberta-nlp/checkpoint-1253/adapter_config.json (deflated 54%)
updating: content/kaim-week-5/lora-xlm-roberta-nlp/checkpoint-1253/adapter_model.safetensors (deflated 7%)
updating: content/kaim-week-5/lora-xlm-roberta-nlp/checkpoint-1253/scheduler.pt (deflated 55%)
updating: content/kaim-week-5/lora-xlm-roberta-nlp/checkpoint-1253/rng_state.pth (deflated 25%)
updating: content/kaim-week-5/lora-xlm-roberta-nlp/checkpoint-1253/special_tokens_map.json (deflated 52%)
updating: conte

In [31]:
from google.colab import files
files.download('/content/kaim-week-5/lora-xlm-roberta-nlp')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
!git add fine_tuning.ipynb

fatal: pathspec 'fine_tuning.ipynb' did not match any files
